In [1]:
import copy

import pandas as pd

In [2]:
df_og = pd.read_pickle("MulTweEmo/datasetPkl/merged_df_with_gold_freq1.pkl")

In [3]:
def anonymize_and_fix_amps(tweet):
    toks = str(tweet).split()
    for idx,t in enumerate(toks):
        if t[0] == '@':
            toks[idx] = '@USER'
        if t == '&amp;':
            toks[idx] = '&'
    return ' '.join(toks)

df_og['tweet'] = df_og['tweet'].apply(anonymize_and_fix_amps)
df_gold = df_og[df_og.T_Surprise.isnull() == False]
df_gold = df_gold[df_gold.M_gold_multi_label.str.len() != 0]

In [4]:
def remove_neutral_smt(labels):
    new_l=[]
    for e in labels:
        if e != "Something else" and e != "Neutral":
            new_l.append(e)
    return new_l

In [5]:
df_gold['M_gold_multi_label'] = df_gold['M_gold_multi_label'].apply(remove_neutral_smt)
df_gold['T_gold_multi_label'] = df_gold['T_gold_multi_label'].apply(remove_neutral_smt)

In [6]:
len(df_gold)

875

In [7]:
df = df_gold

In [8]:
len(df)

875

In [9]:
def replace_string(row):
    new_row = []
    for item in row:
        new_row.append(item.replace('../',''))
    return new_row

df['path_photos'] = df['path_photos'].apply(replace_string)

In [10]:
columns = ['tweet','path_photos', 'M_gold_multi_label', 'T_gold_multi_label']
df = df.loc[:, columns]

In [11]:
df = df.convert_dtypes()

In [12]:
df = df.explode(['path_photos']).reset_index(drop=True)

In [13]:
df = df[:50]

In [14]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import tensorflow as tf

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda:0" if torch.cuda.is_available() else "cpu" 

def extract_and_concatenate(curr_txt, curr_img):
    curr_img = Image.open(curr_img)
    inputs = processor(text=curr_txt, 
                       images=curr_img, 
                       return_tensors='pt', 
                       padding='max_length', 
                       truncation=True)
    inputs = inputs
    outputs = model(**inputs)
    image_features = outputs.image_embeds
    text_features = outputs.text_embeds
    
    concat = torch.cat((image_features, text_features), dim=1)
    #concat = tf.concat(values=[image_features, text_features], axis=1)
    return concat

images = df.path_photos.values
texts = df.tweet.values

X = []

for idx, p in enumerate(images):
    curr_img = images[idx]
    curr_txt = texts[idx]
    print(idx)
    conc = extract_and_concatenate(curr_txt, curr_img)
    X.append(conc)


2024-02-04 20:01:33.274436: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 20:01:33.274558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 20:01:33.325979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 20:01:33.445444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-04 20:01:35.237619: W tensorflow/compiler/tf2

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [18]:
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer

X = np.squeeze(np.asarray([i.detach().numpy() for i in X]))

y = df['T_gold_multi_label']

mlb = MultiLabelBinarizer()
yt = mlb.fit_transform(y)

In [20]:
len(X), len(yt)

(50, 50)

In [23]:
from keras import backend as K
from keras.layers import Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, schedules, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM
import tensorflow as tf 

tf.config.run_functions_eagerly(True)

#keras custom f1_score metric
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric."""

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric."""

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
        
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, yt, test_size=0.15,
                                                   shuffle=True, random_state=42)

model_1 = Sequential(name="ANN-1")
model_1.add(Input(shape=(1024,), name='input'))  # is 512 when using only visual or textual embeddings
                                 # & 1024 when using multimodal embeddings
model_1.add(Dense(400, activation='relu', name="feedforward_1"))
model_1.add(Dropout(0.4, name='dropout_0.4'))

model_1.add(Dense(200, activation='relu', name="feedforward_2"))
model_1.add(Dropout(0.2, name="dropout_0.2"))
model_1.add(Dense(8, activation='sigmoid', name="output"))
model_1.output_shape

#opt = tf.keras.optimizers.SGD(learning_rate=0.002, nesterov=True)
model_1.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=[f1, "accuracy"],)

In [24]:
es = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
BATCH = int(len(X_train)/10)

print('Train...')
model_1.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=300,
          validation_split=0.15,
          shuffle=True,
          callbacks=[es]
          )

Train...


/home/picomachine/.virtualenvs/thesis/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/300


2024-02-04 21:11:11.029415: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:504] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-12.2
  /usr/local/cuda
  /home/picomachine/.virtualenvs/thesis/lib/python3.10/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/picomachine/.virtualenvs/thesis/lib/python3.10/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.
2024-02-04 21:11:11.186616: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-04 2

UnknownError: Exception encountered when calling layer 'output' (type Dense).

{{function_node __wrapped__Sigmoid_device_/job:localhost/replica:0/task:0/device:GPU:0}} JIT compilation failed. [Op:Sigmoid] name: 

Call arguments received by layer 'output' (type Dense):
  • inputs=tf.Tensor(shape=(4, 200), dtype=float32)

In [0]:
predict= model_1.evaluate(x=X_test, y=y_test)

print("Keras F1")
print("ANN-1:")
print("Loss:"+str(predict[0]))
print("F1:"+str(predict[1]))
print("Accuracy:"+str(predict[2]))

from sklearn.metrics import f1_score

y_pred = model_1.predict([X_test], verbose=3)

result_1 = f1_score(y_true=y_test.round(), y_pred=y_pred.round(), labels=None, average="weighted")

from sklearn.metrics import classification_report
labels=mlb.classes_
print(classification_report(y_true=y_test.round(), y_pred=y_pred.round(), target_names=labels))